In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import math

from cmdstanpy import CmdStanModel

Below we repeat the analysis that was done in the original simulation.

In [2]:
chicks = pd.read_table("chickens.dat", sep="\\s+")
chicks["exposed_est"] -= 1
chicks["sham_est"] -= 1
chicks.head()

,freq,sham_n,sham_est,sham_se,exposed_n,exposed_est,exposed_se
0,1,32,-0.005,0.041,32,0.036,0.041
1,15,32,0.013,0.042,36,0.173,0.034
2,30,32,0.033,0.032,32,0.107,0.035
3,45,32,-0.010,0.032,32,0.181,0.052
4,60,32,-0.002,0.040,32,0.136,0.044


In [141]:
chick_model = CmdStanModel(stan_file="dynamic_hier.stan")
chick_data = {
    "num_expts": len(chicks),
    "avg_treated_response": chicks["exposed_est"],
    "avg_control_response": chicks["sham_est"],
    "treated_se": chicks["exposed_se"],
    "control_se": chicks["sham_se"],
    "expt_id": list(range(1, len(chicks) + 1)),
}
chick_fit = chick_model.sample(data=chick_data, adapt_delta=0.9)

15:07:42 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

15:07:42 - cmdstanpy - INFO - CmdStan done processing.
15:07:42 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: normal_lpdf: Location parameter[1] is inf, but must be finite! (in 'dynamic_hier.stan', line 28, column 4 to column 57)
	Exception: normal_lpdf: Location parameter[1] is inf, but must be finite! (in 'dynamic_hier.stan', line 28, column 4 to column 57)
Consider re-running with show_console=True if the above output is unclear!


The function below simulates a dataset for `num_experiments` experiments, all assumed to have the same proportion `prop_treatment` allocated to the treatment group. Here one complication is that the original simulation assumes all experiments have the same number of subjects in the control and treatment groups.

The original assumptions are:

$$
\begin{align*}
    y_{j1} \mid \theta_j, b_j &\sim N(\theta_j + b_j, s_{j1}) \\
    y_{j0} \mid b_j &\sim N(b_j, s_{j0}) \\
    \theta_j &\sim N(\mu_\theta, \sigma_\theta) \\
    b_j &\sim N(\mu_b, \sigma_b)
\end{align*}
$$

These assumptions mostly still work here since $s_{j1}$ and $s_{j0}$ need not be the same for all experiments $j$. However, in the original simulation, it was assumed that $s_{j1} = s_{j0} = 0.04$ for all $j$. This is reasonable if the treatment and control groups have the same size and if we expect the treatment/sham effects to have similar spread, which was empirically shown for the chicken dataset. 

However, if the groups have unequal numbers of subjects, then we expect the group with fewer subjects to have higher variance. We let $Y_{j0}^{(i)}$ and $Y_{j1}^{(k)}$ be the outcomes for the $i$-th and $k$-th individuals in the control and treatment groups respectively for experiment $j$. Let the $j$-th experiment have $N_{j0}$ and $N_{j1}$ subjects in control and treatment groups respectively. If the number of subjects in each group is at least 30, then assuming the individual outcomes are iid, the CLT gives

$$ y_{j1} = \frac{1}{N_{j1}} \sum_{k=1}^{N_{j1}} Y_{j1}^{(k)} \rightarrow N(\mu_{j1}, \sigma_{j1}) $$

where $\sigma_{j1} = \sqrt{\frac{\text{var}(Y_{j1})}{N_{j1}}}$ and similarly for $y_{j0}$. We identify $\mu_{j1}$ with $\theta_j + b_j$ and $\sigma_{j1}$ with $s_{j1}$. Since $s_{j1} = 0.04$ in the original simulation, we use $\text{var}(Y_{j1}) = 32 \times (0.04)^2$ for all experiments $j$; likewise for $\text{var}(Y_{j0})$.

*Thought: What if the sample sizes are too small for the CLT? Maybe bootstrap or make distributional assumptions on $Y_{j1}^{(k)}$?*

In [4]:
def fake_expts_with_prop(
    num_subjects_per_expt,
    prop_treatment,
    mu_b,
    mu_theta,
    sigma_b,
    sigma_theta,
    sigma_treatment,
    sigma_control,
):
    """
    Generate synthetic data with a specified proportion of treated subjects.

    Args:
    num_subjects_per_expt: A list of integers, the number of subjects in each
        experiment.
    prop_treatment: A float, the proportion of subjects that are treated.
    mu_b: A float, the true (mean) of the control group response.
    mu_theta: A float, the true (mean) treatment effect.
    sigma_b: A float, the standard deviation of b_j.
    sigma_theta: A float, the standard deviation of theta_j.
    sigma_treatment: A float, the standard deviation of the treated group
        response.
    sigma_control: A float, the standard deviation of the control group
        response.
    """
    assert len(num_subjects_per_expt) == len(prop_treatment)
    assert sigma_theta >= 0 and sigma_b >= 0
    assert sigma_treatment >= 0 and sigma_control >= 0

    num_expts = len(num_subjects_per_expt)
    num_treated = np.floor(prop_treatment * num_subjects_per_expt).astype(int)
    num_control = num_subjects_per_expt - num_treated

    sigma_y1 = sigma_treatment / np.sqrt(num_treated)
    sigma_y0 = sigma_control / np.sqrt(num_control)

    theta = np.random.normal(mu_theta, sigma_theta, num_expts)
    b = np.random.normal(mu_b, sigma_b, num_expts)

    return {
        "true_params": {
            "mu_b": mu_b,
            "mu_theta": mu_theta,
            "sigma_b": sigma_b,
            "sigma_theta": sigma_theta,
            "sigma_treatment": sigma_treatment,
            "sigma_control": sigma_control,
        },
        "num_expts": len(num_subjects_per_expt),
        "avg_treated_response": np.random.normal(theta + b, sigma_y1),
        "avg_control_response": np.random.normal(b, sigma_y0),
        "treated_se": sigma_y1,
        "control_se": sigma_y0,
        "expt_id": list(range(1, num_expts + 1)),
    }

In [165]:
def get_model_inferences(data, estimate, se, conf_lower, conf_upper):
    # an observation is significant if 0 is not in the interval
    significant = ~((conf_lower < 0) & (0 < conf_upper))
    correct_sign = np.sign(data["true_params"]["mu_theta"]) == np.sign(estimate)
    error = data["true_params"]["mu_theta"] - estimate

    return pd.DataFrame(
        {
            "estimate": estimate,
            "se": se,
            "conf_lower": conf_lower,
            "conf_upper": conf_upper,
            "is_signif": significant,
            "correct_sign": correct_sign,
            "error": error,
        }
    )


def get_exposed_only_inferences(data, alpha=0.05):
    z_value = stats.norm.ppf(1 - alpha / 2)
    estimate = data["avg_treated_response"]
    se = data["treated_se"]
    conf_lower = estimate - z_value * se
    conf_upper = estimate + z_value * se

    return get_model_inferences(data, estimate, se, conf_lower, conf_upper)


def get_difference_inferences(data, alpha=0.05):
    z_value = stats.norm.ppf(1 - alpha / 2)
    estimate = data["avg_treated_response"] - data["avg_control_response"]
    se = np.sqrt(data["treated_se"] ** 2 + data["control_se"] ** 2)
    conf_lower = estimate - z_value * se
    conf_upper = estimate + z_value * se

    return get_model_inferences(data, estimate, se, conf_lower, conf_upper)


def get_posterior_inferences(model, data, alpha=0.05):
    fit = model.sample(data=data, iter_sampling=1000, show_progress=False)
    estimate = np.mean(fit.stan_variable("theta"), axis=0)
    se = np.std(fit.stan_variable("theta"), axis=0)

    thetas = fit.stan_variable("theta")
    conf_lower = np.quantile(thetas, alpha / 2, axis=0)
    conf_upper = np.quantile(thetas, 1 - alpha / 2, axis=0)

    return get_model_inferences(data, estimate, se, conf_lower, conf_upper)


def get_summary(df):
    prop_signif = np.mean(df["is_signif"])
    mse = np.mean(df["error"] ** 2)
    type_s_rate = (
        len(df[df["is_signif"] & ~df["correct_sign"]]) / len(df) if len(df) > 0 else 0
    )
    return {
        "prop_signif": prop_signif,
        "type_s_rate": type_s_rate,
        "mse": mse,
    }

In [171]:
def repeat_inferences(
    model,
    num_repetitions,
    num_subjects_per_expt,
    prop_treatment,
    mu_b,
    mu_theta,
    sigma_b,
    sigma_theta,
    sigma_treatment,
    sigma_control,
    show_progress=False,
):
    summaries = {
        method: {"prop_signif": [], "type_s_rate": [], "mse": []}
        for method in ["exposed_only", "difference", "posterior"]
    }

    for i in range(num_repetitions):
        fake_data = fake_expts_with_prop(
            num_subjects_per_expt,
            prop_treatment,
            mu_b,
            mu_theta,
            sigma_b,
            sigma_theta,
            sigma_treatment,
            sigma_control,
        )

        exposed_only = get_exposed_only_inferences(fake_data)
        difference = get_difference_inferences(fake_data)
        posterior = get_posterior_inferences(model, fake_data)

        for method, df in zip(
            ["exposed_only", "difference", "posterior"],
            [exposed_only, difference, posterior],
        ):
            summary = get_summary(df)
            for key in summary:
                summaries[method][key].append(summary[key])

        if show_progress and i % 10 == 0:
            print(f"Completed repetition {i} of {num_repetitions - 1}")

    return summaries

In [155]:
num_expts = 38
num_subjects_per_expt = np.repeat(64, num_expts)

const_prop = np.linspace(0.5, 0.05, 38)  # constant proportion treated
varying_prop = np.linspace(0.5, 0.95, 38)  # increase from 0.5 to 1 linearly

In [169]:
const_prop_results = repeat_inferences(
    model=CmdStanModel(stan_file="dynamic_hier.stan"),
    num_repetitions=200,
    num_subjects_per_expt=num_subjects_per_expt,
    prop_treatment=const_prop,
    mu_b=0,
    mu_theta=chick_fit.theta.mean(),
    sigma_b=np.std(np.mean(chick_fit.b, axis=0)),
    sigma_theta=np.std(np.mean(chick_fit.theta, axis=0)),
    sigma_treatment=(32**0.5) * 0.04,
    sigma_control=(32**0.5) * 0.04,
    show_progress=True,
)

Completed repetition 1 of 200
Completed repetition 11 of 200
Completed repetition 21 of 200
Completed repetition 31 of 200
Completed repetition 41 of 200
Completed repetition 51 of 200
Completed repetition 61 of 200
Completed repetition 71 of 200
Completed repetition 81 of 200
Completed repetition 91 of 200
Completed repetition 101 of 200
Completed repetition 111 of 200
Completed repetition 121 of 200
Completed repetition 131 of 200
Completed repetition 141 of 200
Completed repetition 151 of 200
Completed repetition 161 of 200
Completed repetition 171 of 200
Completed repetition 181 of 200
Completed repetition 191 of 200


In [170]:
varying_prop_results = repeat_inferences(
    model=CmdStanModel(stan_file="dynamic_hier.stan"),
    num_repetitions=200,
    num_subjects_per_expt=num_subjects_per_expt,
    prop_treatment=varying_prop,
    mu_b=0,
    mu_theta=chick_fit.theta.mean(),
    sigma_b=np.std(np.mean(chick_fit.b, axis=0)),
    sigma_theta=np.std(np.mean(chick_fit.theta, axis=0)),
    sigma_treatment=(32**0.5) * 0.04,
    sigma_control=(32**0.5) * 0.04,
    show_progress=True,
)

Completed repetition 1 of 200
Completed repetition 11 of 200
Completed repetition 21 of 200
Completed repetition 31 of 200
Completed repetition 41 of 200
Completed repetition 51 of 200
Completed repetition 61 of 200
Completed repetition 71 of 200
Completed repetition 81 of 200
Completed repetition 91 of 200
Completed repetition 101 of 200
Completed repetition 111 of 200
Completed repetition 121 of 200
Completed repetition 131 of 200
Completed repetition 141 of 200
Completed repetition 151 of 200
Completed repetition 161 of 200
Completed repetition 171 of 200
Completed repetition 181 of 200
Completed repetition 191 of 200


In [173]:
for method in ["exposed_only", "difference", "posterior"]:
    print(method)
    for key in ["prop_signif", "type_s_rate", "mse"]:
        print(key)
        const_prop_results[method][key] = np.mean(const_prop_results[method][key])
        varying_prop_results[method][key] = np.mean(varying_prop_results[method][key])

        print(f"Constant prop: {const_prop_results[method][key]}")
        print(f"Varying prop: {varying_prop_results[method][key]}")
    print()


exposed_only
prop_signif
Constant prop: 0.42236842105263156
Varying prop: 0.6848684210526315
type_s_rate
Constant prop: 0.0035526315789473684
Varying prop: 0.006973684210526315
mse
Constant prop: 0.007645530621790338
Varying prop: 0.00431726538204074

difference
prop_signif
Constant prop: 0.3434210526315789
Varying prop: 0.34315789473684205
type_s_rate
Constant prop: 0.004210526315789474
Varying prop: 0.004736842105263157
mse
Constant prop: 0.008768083582281384
Varying prop: 0.008204222542839186

posterior
prop_signif
Constant prop: 0.6698684210526317
Varying prop: 0.7760526315789472
type_s_rate
Constant prop: 0.0
Varying prop: 0.0005263157894736842
mse
Constant prop: 0.0016331201410843606
Varying prop: 0.0020920571166903952



In [ ]:
chicks_avg_subjects = math.floor(np.mean(chicks["exposed_n"] + chicks["sham_n"]))
chick_mu_b = chick_fit.stan_variable("b").mean()
chick_mu_theta = chick_fit.stan_variable("theta").mean()
chick_se = (32**0.5) * 0.04

balanced_sim = fake_expts_with_prop(
    num_subjects_per_expt=np.repeat(chicks_avg_subjects, len(chicks)),
    prop_treatment=np.repeat(0.5, len(chicks)),
    mu_b=0,
    mu_theta=chick_mu_theta,
    sigma_b=np.std(np.mean(chick_fit.stan_variable("b"), axis=0)),
    sigma_theta=np.std(np.mean(chick_fit.stan_variable("theta"), axis=0)),
    sigma_treatment=chick_se,
    sigma_control=chick_se,
)